In [4]:
%pip install torch pandas torchvision scikit-learn tqdm kaggle torchmetrics huggingface_hub -q

Note: you may need to restart the kernel to use updated packages.


In [6]:
from huggingface_hub import hf_hub_download
import os

model_files = [
    "mendeley.zip"
]

# Target local directory
local_dir = "."
os.makedirs(local_dir, exist_ok=True)

# Download each model
for filename in model_files:
    hf_hub_download(
        repo_id="omkar334/agri",
        filename=filename,
        local_dir=local_dir,
    )

!unzip -q mendeley.zip -d data3

In [11]:
from dataset import get_dataloaders
train_loader_labeled, val_loader_labeled, train_loader_unlabeled = get_dataloaders('data3/mendeley', 16)

In [12]:
from convnext import Autoencoder
import torch
import torch.nn as nn
import torch.optim as optim


DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model = Autoencoder(11).to(DEVICE)
reconstruction_criterion = nn.MSELoss()  # Unsupervised Loss
classification_criterion = nn.CrossEntropyLoss()  # Supervised Loss
optimizer = optim.Adam(model.parameters(), lr=1e-4)

In [ ]:
from train import train_model, validate_model

model = train_model(
    model,
    train_loader_labeled,
    train_loader_unlabeled,
    reconstruction_criterion,
    classification_criterion,
    optimizer,
    num_epochs=35,
)

Epoch [1/35], Total Loss: 714.1368, Reconstruction Loss: 26.1035, Classification Loss: 688.0333, Accuracy: 0.2678


Epoch [2/35], Total Loss: 663.0012, Reconstruction Loss: 25.3293, Classification Loss: 637.6719, Accuracy: 0.3377


Epoch [3/35], Total Loss: 602.5347, Reconstruction Loss: 25.0346, Classification Loss: 577.5000, Accuracy: 0.4041


Epoch [4/35], Total Loss: 525.2872, Reconstruction Loss: 24.8172, Classification Loss: 500.4700, Accuracy: 0.4809


Epoch [5/35], Total Loss: 394.1712, Reconstruction Loss: 24.6765, Classification Loss: 369.4947, Accuracy: 0.6209


...

Epoch [24/35], Total Loss: 31.6464, Reconstruction Loss: 24.0639, Classification Loss: 7.5826, Accuracy: 0.9911


Epoch [25/35], Total Loss: 31.1832, Reconstruction Loss: 24.0458, Classification Loss: 7.1374, Accuracy: 0.9902


Epoch [26/35], Total Loss: 32.2272, Reconstruction Loss: 24.0435, Classification Loss: 8.1837, Accuracy: 0.9896


Epoch [27/35], Total Loss: 31.3348, Reconstruction Loss: 24.0370, Classification Loss: 7.2978, Accuracy: 0.9894


Epoch 28/35:  73%|███████▎  | 245/337 [01:23<00:30,  2.98it/s]

In [16]:
torch.save(model.state_dict(), 'models/convnext_mendeley.pth')

In [18]:
validate_model(model, val_loader_labeled, 11)

Validation Accuracy: 0.6914
Validation Precision: 0.7020
Validation Recall: 0.6241
Validation F1 Score: 0.6460
Number of samples validated on: 256


In [15]:
import time